In [1]:
# add_zip_from_blockgroup_autofetch.py
# ---------------------------------------------------
# Runs in the SAME FOLDER as your CSV.
# Input : Howard_demographics_block_2022-2023.csv (needs 'county_name')
# Output: Howard_demographics_block_2022-2023_with_zip.csv
#
# What it does if no local crosswalk is present:
#   1) Downloads Census tract↔ZCTA relationship file (2020 preferred; 2010 fallback).
#   2) Filters to Maryland (STATE=24), builds 11-digit tract GEOID.
#   3) Picks dominant ZCTA per tract (based on largest available weight column).
#   4) Joins your Block Group rows (via tract) to add a ZIP column.

import os
import re
import sys
import io
import pandas as pd

# ---------- Filenames (current folder) ----------
IN_CSV  = "Howard_demographics_block_2022-2023.csv"
OUT_CSV = "Howard_demographics_block_2022-2023_with_zip.csv"

# Optional local crosswalks (if you’ve already got them)
LOCAL_BG = ["bg_to_zcta_md.csv", "bg_to_zcta_us.csv"]
LOCAL_TR = ["tract_to_zcta_md.csv", "tract_to_zcta_us.csv"]

# Known Census relationship file URLs (2020 preferred, 2010 fallback)
REMOTE_RELATIONSHIP_SOURCES = [
    # 2020 relationship (Excel). This is the current preferred file.
    "https://www2.census.gov/geo/docs/maps-data/data/rel/2020/zcta_tract_rel_20.xlsx",
    # 2010 relationship (TXT/CSV-like)
    "https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_tract_rel_10.txt",
]

# FIPS constants
STATE_FIPS_MD = "24"   # Maryland

# -------- Helpers --------
TRACT_RE = re.compile(
    r"Block\s*Group\s*(?P<bg>\d+)\s*;\s*Census\s*Tract\s*(?P<tract>[0-9.]+)\s*;\s*(?P<county>[^;]+)\s*;\s*(?P<state>.+)$",
    re.IGNORECASE
)

def tract_to_6d(tract_str: str) -> str:
    """Convert '6011.03' → '601103', '6012' → '601200'"""
    if not isinstance(tract_str, str):
        return None
    ts = tract_str.strip()
    if "." in ts:
        left, right = ts.split(".", 1)
        left = re.sub(r"\D", "", left)
        right = re.sub(r"\D", "", right)
        return f"{left.zfill(4)}{right.zfill(2)}"
    else:
        left = re.sub(r"\D", "", ts)
        return f"{left.zfill(4)}00"

def parse_county_name(val: str):
    """Parse your county_name string into bg, tract6, county, state."""
    if not isinstance(val, str):
        return {"bg": None, "tract6": None, "county": None, "state": None}
    m = TRACT_RE.search(val.strip())
    if not m:
        return {"bg": None, "tract6": None, "county": None, "state": None}
    return {
        "bg": m.group("bg").strip(),
        "tract6": tract_to_6d(m.group("tract")),
        "county": m.group("county").strip(),
        "state": m.group("state").strip(),
    }

def find_first_existing(names):
    for nm in names:
        if os.path.exists(nm):
            return nm
    return None

def choose_weight_col(df):
    for c in ["POPPT", "AREAPT", "AREALANDPT", "RES_RATIO", "BUS_RATIO", "pop_share", "aland_share", "area_share", "weight", "share"]:
        if c in df.columns:
            return c
    return None

def normalize_zcta_col(df):
    # Try multiple common ZCTA column names → "zcta5"
    for c in ["ZCTA5", "ZCTA5CE10", "ZCTA5CE20", "ZCTA", "zcta5", "zcta"]:
        if c in df.columns:
            df = df.rename(columns={c: "zcta5"})
            break
    if "zcta5" in df.columns:
        df["zcta5"] = df["zcta5"].astype(str).str.zfill(5)
    return df

def add_tract_geoid11(df):
    """Create 11-digit tract GEOID column 'tract_geoid11' from parts, if needed."""
    # If already present under some common names, normalize to 'tract_geoid11'
    for c in ["GEOID", "GEOID10", "GEOID20", "TRACT_GEOID", "tract_geoid11"]:
        if c in df.columns:
            df = df.rename(columns={c: "tract_geoid11"})
            df["tract_geoid11"] = df["tract_geoid11"].astype(str).str.zfill(11)
            return df

    # Else try building from parts
    state_cand  = None
    county_cand = None
    tract_cand  = None
    for c in ["STATE", "STATEFP", "STATEFP10", "STATEFP20"]:
        if c in df.columns: state_cand = c; break
    for c in ["COUNTY", "COUNTYFP", "COUNTYFP10", "COUNTYFP20"]:
        if c in df.columns: county_cand = c; break
    for c in ["TRACT", "TRACTCE", "TRACTCE10", "TRACTCE20"]:
        if c in df.columns: tract_cand = c; break

    if state_cand and county_cand and tract_cand:
        df["tract_geoid11"] = (
            df[state_cand].astype(str).str.zfill(2) +
            df[county_cand].astype(str).str.zfill(3) +
            df[tract_cand].astype(str).str.zfill(6)
        )
    else:
        raise ValueError("Could not identify columns to build 'tract_geoid11' from the relationship file.")
    return df

def load_relationship_crosswalk():
    """
    Try:
      1) local block-group crosswalk,
      2) local tract crosswalk,
      3) download 2020/2010 tract↔ZCTA from Census.
    Return a DataFrame with columns: tract_geoid11, zcta5, and a weight col (optional).
    """
    # 1) Local BG crosswalk (rare): we can collapse to tract if needed, but we’ll prefer tract level uniformly
    bg_local = find_first_existing(LOCAL_BG)
    if bg_local:
        xw = pd.read_csv(bg_local, dtype=str, encoding="utf-8-sig")
        xw = normalize_zcta_col(xw)
        # If it’s truly BG-level with 'bg_geoid', reduce to tract by slicing first 11 chars
        if "bg_geoid" in xw.columns and "tract_geoid11" not in xw.columns:
            xw["tract_geoid11"] = xw["bg_geoid"].astype(str).str.slice(0, 11)
        if "tract_geoid11" not in xw.columns:
            raise ValueError(f"{bg_local} is missing tract info; expected 'bg_geoid' or 'tract_geoid11'.")
        return xw

    # 2) Local tract crosswalk
    tr_local = find_first_existing(LOCAL_TR)
    if tr_local:
        tx = pd.read_csv(tr_local, dtype=str, encoding="utf-8-sig")
        tx = normalize_zcta_col(tx)
        if "tract_geoid11" not in tx.columns:
            # Try to infer from a generic 'tract' column
            if "tract" in tx.columns:
                tx["tract_geoid11"] = tx["tract"].astype(str).str.zfill(11)
            else:
                raise ValueError(f"{tr_local} must have 'tract_geoid11' or 'tract' column.")
        return tx

    # 3) Download from Census (2020 preferred, 2010 fallback)
    import requests
    last_err = None
    for url in REMOTE_RELATIONSHIP_SOURCES:
        try:
            print(f"Downloading relationship file: {url}")
            resp = requests.get(url, timeout=60)
            resp.raise_for_status()

            if url.endswith(".xlsx"):
                rel = pd.read_excel(io.BytesIO(resp.content), dtype=str)
            else:
                # 2010 TXT is pipe or comma-delimited; let pandas sniff
                rel = pd.read_csv(io.StringIO(resp.text), dtype=str)

            # Normalize columns
            rel = normalize_zcta_col(rel)
            rel = add_tract_geoid11(rel)

            # Filter to Maryland (STATE 24) if a state column exists; otherwise filter by tract prefix
            state_col = None
            for c in ["STATE", "STATEFP", "STATEFP10", "STATEFP20"]:
                if c in rel.columns: state_col = c; break

            if state_col:
                rel_md = rel[rel[state_col].astype(str).str.zfill(2) == STATE_FIPS_MD].copy()
            else:
                # Filter by tract_geoid11 prefix
                rel_md = rel[rel["tract_geoid11"].astype(str).str.startswith(STATE_FIPS_MD)].copy()

            # Keep essential columns
            cols_keep = ["tract_geoid11", "zcta5"]
            # Add a weight if present
            wcol = choose_weight_col(rel_md)
            if wcol and wcol not in cols_keep:
                cols_keep.append(wcol)
            rel_md = rel_md[cols_keep].dropna(subset=["tract_geoid11", "zcta5"])

            # If multiple ZCTAs per tract, keep the dominant by weight (or first if none)
            if wcol:
                rel_md = (rel_md.sort_values(wcol, ascending=False)
                                  .drop_duplicates(subset=["tract_geoid11"], keep="first"))
            else:
                rel_md = rel_md.drop_duplicates(subset=["tract_geoid11"], keep="first")

            return rel_md.reset_index(drop=True)

        except Exception as e:
            last_err = e
            print(f"  -> Failed to use {url}: {e}")

    raise RuntimeError(f"Could not obtain a tract↔ZCTA relationship file. Last error: {last_err}")

# -------- Main --------
def main():
    if not os.path.exists(IN_CSV):
        sys.exit(f"Input file not found: {IN_CSV}")

    df = pd.read_csv(IN_CSV, dtype=str, encoding="utf-8-sig")
    if "county_name" not in df.columns:
        sys.exit("The input CSV must contain a 'county_name' column.")

    parsed = df["county_name"].apply(parse_county_name).apply(pd.Series)
    df = pd.concat([df, parsed], axis=1)

    # Keep Maryland/Howard rows (your file already is, but it's safe)
    md_mask = df["state"].str.contains("Maryland", case=False, na=False)
    df = df.loc[md_mask].copy()

    # Build tract GEOID (11 digits): 24 + 027 + tract6  (Howard county = 027)
    df["tract_geoid11"] = "24" + "027" + df["tract6"].fillna("")

    # Load or fetch relationship crosswalk (tract→ZCTA)
    rel = load_relationship_crosswalk()

    # Merge and write
    out = df.merge(rel[["tract_geoid11", "zcta5"]], on="tract_geoid11", how="left")
    out = out.rename(columns={"zcta5": "zip"})
    out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

    total = len(out)
    mapped = out["zip"].notna().sum()
    print(f"✅ Wrote: {OUT_CSV}")
    print(f"ZIP coverage: {mapped}/{total} rows mapped.")
    if mapped < total:
        print("Note: Some tracts can map to multiple ZCTAs; we chose the dominant one by weight if available.")

if __name__ == "__main__":
    main()


  -> Failed to use https://www2.census.gov/geo/docs/maps-data/data/rel/2020/zcta_tract_rel_20.xlsx: 404 Client Error: Not Found for url: https://www2.census.gov/geo/docs/maps-data/data/rel/2020/zcta_tract_rel_20.xlsx
✅ Wrote: Howard_demographics_block_2022-2023_with_zip.csv
ZIP coverage: 276/332 rows mapped.
Note: Some tracts can map to multiple ZCTAs; we chose the dominant one by weight if available.
